# Segmenting and Clustering Neighborhoods in Toronto

<div id='part-1'/>

____
## Part 1 - Data Scraping

In [1]:
from bs4 import BeautifulSoup
import urllib3.request
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium
import os
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors


In [2]:
page_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
proxy_url = ""

if proxy_url.strip() != "":
    http = urllib3.ProxyManager(proxy_url)
else:
    http = urllib3.PoolManager()

req = http.request('GET', page_url)
soup = BeautifulSoup(req.data, 'html.parser')


/home/jupyterlab/conda/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [3]:
# locate postcode table
toronto_table = soup.find('table',{'class':'wikitable sortable'})

# process table rows and build raw_df
raw_df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
rows = toronto_table.findAll('tr')
for row in rows:
    row_items = row.findAll('td')
    if len(row_items) > 0:
        postcode = row_items[0].text.strip()
        borough = row_items[1].text.strip()
        if borough.lower() != "not assigned":
            neighborhood = row_items[2].text.strip()
            raw_df = raw_df.append({'PostalCode':postcode, 
                                    'Borough':borough, 
                                    'Neighborhood':neighborhood}, 
                                   ignore_index = True)

raw_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [4]:
#Combine Neighbourhoods belonging to the same borough in one row.
#Replace 'Not Assigned' neighborhoods with Borough's name

grouped = []
for name, group in raw_df.groupby(['PostalCode', 'Borough'])['Neighborhood']:
    nblist = ''.join(str(x) + ", " for x in group.tolist()).strip(", ")
    if nblist == "Not assigned":
        nblist = name[1]
    grouped.append((name[0], name[1], nblist))

toronto_df = pd.DataFrame(grouped, columns=['PostalCode', 'Borough', 'Neighborhood'])
print(toronto_df.shape)
toronto_df.head(10)

(103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [5]:
toronto_df.tail(10)

,PostalCode,Borough,Neighborhood
93,M9A,Etobicoke,Islington Avenue
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery, Humberlea"
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


In [6]:
# Verify is there any record with neighbourhood as not assigned, it's expected to be empty.
toronto_df.query("Neighborhood == 'Not assigned'")

,PostalCode,Borough,Neighborhood


In [7]:
# verify a known 'Not assigned' Neighborhood case, it should be equal to Borough. 
toronto_df.query("PostalCode == 'M7A'") 

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [8]:
# check number of rows in dataframe using 'shape'
toronto_df.shape

(103, 3)

___
## Part 2 : Geocoding

In [9]:
import csv
with open('Geospatial_Coordinates.csv', 'rt') as geo_file:
    geo_reader = csv.reader(geo_file, delimiter=',')
    for row in geo_reader:
        #print(', '.join(row))
        toronto_df.loc[toronto_df['PostalCode'] == row[0], 'Latitude'] = float(row[1])        
        toronto_df.loc[toronto_df['PostalCode'] == row[0], 'Longitude'] = float(row[2])
        
toronto_df.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [11]:
toronto_df.tail(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
93,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
94,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724
95,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol...",43.643515,-79.577201
96,M9L,North York,Humber Summit,43.756303,-79.565963
97,M9M,North York,"Emery, Humberlea",43.724766,-79.532242
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
102,M9W,Etobicoke,Northwest,43.706748,-79.594054


In [12]:
toronto_df.shape

(103, 5)

___
## Part 3 - Neighborhoods Clustering
  

In [15]:
# Select Toronto boroughs with word "Toronto" in the name.
toronto_df = toronto_df[toronto_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
print(toronto_df.shape)
toronto_df.head(10)

(38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [16]:
toronto_df.tail(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
28,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
29,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564
31,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
32,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
33,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
34,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
35,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
36,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
37,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558


In [17]:
# Build Toronto map including markers for boroughs.
address = 'Toronto'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} is {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Toronto is 43.653963, -79.387207.


In [18]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], \
                                           toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto_map)  
    
toronto_map

In [51]:
# Foursquare ID and Secret masked as xxx for security purpose
CLIENT_ID = 'xxx'
CLIENT_SECRET = 'xxx'
VERSION = '20180605' # Foursquare API version

print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: xxx
CLIENT_SECRET:xxx


In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
        
        LIMIT = 100 
        radius = 500    
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            print("ERROR: ", url)

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
# Get venues for every neighborhood
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

print(toronto_venues.shape)
toronto_venues.head()

(1697, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


In [35]:
# Build venue categories dataframe
# Group by neighborhood and calculate mean value for each

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.head()

# add neighborhood column back to dataframe and move column to the first column
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
col_index = toronto_onehot.columns.tolist().index('Neighborhood')
col_order = [toronto_onehot.columns[col_index]] \
                + list(toronto_onehot.columns[0:col_index]) \
                + list(toronto_onehot.columns[col_index+1:])
toronto_onehot = toronto_onehot[col_order]
print("categories dataset shape {}".format(toronto_onehot.shape))
toronto_onehot.head()

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print("categories grouped by neighborhood shape {}".format(toronto_grouped.shape))
toronto_grouped.head(10)

categories dataset shape (1697, 238)
categories grouped by neighborhood shape (38, 238)


,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.0,0.0,0.0,0.010000,0.000000,0.000000,0.010000,0.000000,0.01,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.062500
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,...,0.0,0.0,0.0,0.012048,0.000000,0.000000,0.012048,0.000000,0.00,0.012048
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.050000,0.000000,0.050000,0.010000,0.000000,0.00,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Church and Wellesley,0.011628,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,...,0.0,0.0,0.0,0.000000,0.011628,0.011628,0.000000,0.011628,0.00,0.011628


In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
# Build ten top venues dataset

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head(10)

(38, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,American Restaurant,Clothing Store,Gym,Hotel,Bakery,Bar
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Bakery,Pub,Farmers Market,Cheese Shop,Seafood Restaurant,Beer Bar,Italian Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Coffee Shop,Café,Burrito Place,Stadium,Bar,Caribbean Restaurant,Furniture / Home Store,Climbing Gym,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Garden Center,Pizza Place,Park,Recording Studio,Restaurant,Burrito Place,Skate Park,Brewery
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Airport Gate,Airport Food Court,Airport
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Park,Bakery,Italian Restaurant,Café,Pub,Pizza Place,Pet Store,Breakfast Spot
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Ice Cream Shop,Indian Restaurant,Bubble Tea Shop,Spa,Salad Place
7,"Chinatown, Grange Park, Kensington Market",Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Bakery,Coffee Shop,Chinese Restaurant,Mexican Restaurant,Gaming Cafe
8,Christie,Café,Grocery Store,Park,Convenience Store,Italian Restaurant,Baby Store,Restaurant,Athletics & Sports,Nightclub,Diner
9,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Restaurant,Burger Joint,Café,Gastropub,Fast Food Restaurant,Mediterranean Restaurant


In [40]:
# Calculate clustering using k-means algorithm

kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2,
       2, 2, 4, 2, 2, 1, 3, 2, 2, 2, 2, 2, 2, 0, 2, 2], dtype=int32)

In [42]:
# Build cluster dataset and plot the map

toronto_merged = toronto_df
# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(toronto_merged.shape)
toronto_merged.head(10) # check the last columns!

(38, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Pub,Coffee Shop,Asian Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Fish & Chips Shop,Falafel Restaurant,Dog Run,Event Space
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Fruit & Vegetable Store,Bakery,Juice Bar,Spa
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,2,Park,Gym,Pub,Brewery,Fast Food Restaurant,Fish & Chips Shop,Liquor Store,Burger Joint,Burrito Place,Sandwich Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Park,Seafood Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Food & Drink Shop,Gym,Burger Joint,Park,Hotel,Sandwich Place,Breakfast Spot,Farmers Market,Donut Shop,Falafel Restaurant
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Bagel Shop,Chinese Restaurant,Dessert Shop,Diner,Rental Car Location,Salon / Barbershop
7,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Pizza Place,Sandwich Place,Dessert Shop,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Thai Restaurant,Seafood Restaurant,Toy / Game Store
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2,Playground,Gym,Tennis Court,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,2,Pub,Coffee Shop,Pizza Place,Light Rail Station,Supermarket,Bagel Shop,Sports Bar,Sushi Restaurant,American Restaurant,Vietnamese Restaurant


In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], \
                                  toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [44]:
# Cluster 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, \
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,0,Coffee Shop,Café,Park,Bakery,Restaurant,Pub,Mexican Restaurant,Breakfast Spot,Theater,Chocolate Shop
35,West Toronto,0,Gift Shop,Breakfast Spot,Restaurant,Burger Joint,Bar,Bank,Movie Theater,Eastern European Restaurant,Italian Restaurant,Dog Run


In [45]:
# Cluster 2

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, \
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,1,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Ice Cream Shop,Indian Restaurant,Bubble Tea Shop,Spa,Salad Place
27,Downtown Toronto,1,Airport Lounge,Airport Terminal,Airport Service,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Airport Gate,Airport Food Court,Airport


In [46]:
# Cluster 3

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, \
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,2,Pub,Coffee Shop,Asian Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Fish & Chips Shop,Falafel Restaurant,Dog Run,Event Space
1,East Toronto,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Fruit & Vegetable Store,Bakery,Juice Bar,Spa
2,East Toronto,2,Park,Gym,Pub,Brewery,Fast Food Restaurant,Fish & Chips Shop,Liquor Store,Burger Joint,Burrito Place,Sandwich Place
3,East Toronto,2,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Park,Seafood Restaurant,Sandwich Place,Cheese Shop
4,Central Toronto,2,Park,Swim School,Dim Sum Restaurant,Bus Line,Yoga Studio,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
5,Central Toronto,2,Food & Drink Shop,Gym,Burger Joint,Park,Hotel,Sandwich Place,Breakfast Spot,Farmers Market,Donut Shop,Falafel Restaurant
6,Central Toronto,2,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Bagel Shop,Chinese Restaurant,Dessert Shop,Diner,Rental Car Location,Salon / Barbershop
7,Central Toronto,2,Pizza Place,Sandwich Place,Dessert Shop,Italian Restaurant,Café,Sushi Restaurant,Coffee Shop,Thai Restaurant,Seafood Restaurant,Toy / Game Store
8,Central Toronto,2,Playground,Gym,Tennis Court,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
9,Central Toronto,2,Pub,Coffee Shop,Pizza Place,Light Rail Station,Supermarket,Bagel Shop,Sports Bar,Sushi Restaurant,American Restaurant,Vietnamese Restaurant


In [47]:
# Cluster 4

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, \
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Downtown Toronto,3,Coffee Shop,Restaurant,Café,Cocktail Bar,Seafood Restaurant,Pub,Hotel,Beer Bar,Italian Restaurant,Bakery


In [48]:
# Cluster 5

toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, \
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Central Toronto,4,Coffee Shop,Café,Sandwich Place,Pizza Place,BBQ Joint,History Museum,Indian Restaurant,Jewish Restaurant,Flower Shop,Liquor Store
